## 最終課題

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse
from collections import deque
import re

# ==============================================================================
# 初期設定
# ==============================================================================

# アクセスしたいWebサイトのURLを指定
BASE_URL = 'https://www.musashino-u.ac.jp/'
# Webサイトへの負荷軽減のためのディレイ時間（秒）
SLEEP_TIME = 1.0 # 負荷軽減のため、1.0秒に設定
# User-Agentを設定
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
# 探索する最大ページ数 (無限ループを防ぐため、非常に大きな値を設定)
MAX_PAGES = 1000000 

# 収集結果を格納する辞書: {URL: <title>}
site_map = {}
# 探索済みのURLを格納するセット (重複アクセス防止)
visited_urls = set()
# これから探索するURLを格納するキュー (幅優先探索用)
url_queue = deque([BASE_URL])
# ベースドメインの取得
base_domain = urlparse(BASE_URL).netloc

# ==============================================================================
# クローリング関数
# ==============================================================================

def get_title_and_links(url):
    """
    指定されたURLにアクセスし、タイトルとページ内の同一ドメインのリンクを抽出する
    """
    # 訪問済みリストにURLを追加し、即座にチェック (無限ループ回避)
    if url in visited_urls:
        return []
    
    # URLにアクセスする前にディレイをかける (負荷軽減)
    time.sleep(SLEEP_TIME)
    
    # 探索ページ数/制限ページ数を表示
    print(f"\n-> アクセス中 ({len(site_map) + 1}ページ目): {url}")
    
    try:
        # HTTPリクエストを送信 (タイムアウト設定)
        res = requests.get(url, headers=HEADERS, timeout=15)
        
        # ステータスコードの確認 (200 OK以外はスキップ)
        if res.status_code != 200:
            print(f"--- 🚫 エラー: ステータスコード {res.status_code} ({res.reason}) - {url}")
            # エラーページも訪問済みとする
            visited_urls.add(url)
            return []
        
        # レスポンスのエンコーディングを設定
        res.encoding = res.apparent_encoding
        
        # HTML解析
        soup = BeautifulSoup(res.text, 'html.parser')
        
        # <title>タグの抽出
        title_tag = soup.find('title')
        title = title_tag.string.strip() if title_tag and title_tag.string else "タイトルなし"
        
        # site_mapに保存
        # 処理成功時のみvisited_urlsに追加し、site_mapを更新
        site_map[url] = title
        visited_urls.add(url)
        print(f"--- ✅ 抽出完了: タイトル='{title}'")
        
        # <a>タグからのリンク抽出
        new_links = []
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            
            # リンクを絶対URLに変換
            absolute_url = urljoin(url, href)
            
            # URLのクエリパラメータやフラグメントを取り除き、探索URLを正規化
            parsed_url = urlparse(absolute_url)
            normalized_url = parsed_url.scheme + "://" + parsed_url.netloc + parsed_url.path
            
            # 💡 PDFファイルを除外する処理 💡
            # パスを小文字にしてチェックし、.pdfで終わる場合はスキップ
            if parsed_url.path.lower().endswith('.pdf'):
                # PDFリンクは無視し、次のリンクへ
                continue 

            # URL末尾のスラッシュを統一 (BASE_URL以外は末尾のスラッシュを削除)
            if normalized_url.endswith('/') and normalized_url != BASE_URL:
                 normalized_url = normalized_url[:-1]
            
            # ドメインチェック (同一ドメインのみを対象とする)
            link_domain = urlparse(normalized_url).netloc
            
            # 同一ドメインであり、かつ既に探索済み・キューにない場合
            if link_domain == base_domain:
                if normalized_url not in visited_urls:
                    if normalized_url not in url_queue:
                         new_links.append(normalized_url)
        
        return new_links
    
    except requests.exceptions.Timeout:
        print(f"--- ❌ エラー: タイムアウト - {url}")
    except requests.exceptions.RequestException as e:
        print(f"--- ❌ エラー: リクエスト失敗 - {e} - {url}")
    except Exception as e:
        print(f"--- ❌ エラー: その他のエラー - {e} - {url}")
        
    # エラーが発生した場合も、再試行しないように訪問済みとする
    visited_urls.add(url)
    return []

# ==============================================================================
# クローリング実行 (幅優先探索 - BFS)
# ==============================================================================

print("=========================================================")
print(f"🚀 クローリング開始: ベースURL {BASE_URL}")
print(f"⏰ ディレイ時間: {SLEEP_TIME}秒")
print(f"🔍 ページ制限: 制限なし (理論上 {MAX_PAGES} ページまで)")
print("=========================================================")

# URLキューが空になるか、設定した最大ページ数に達するまでループ
while url_queue and len(site_map) < MAX_PAGES:
    current_url = url_queue.popleft() # キューから次のURLを取り出す
    
    # タイトルと新規リンクを取得
    links = get_title_and_links(current_url)
    
    # 取得した新規リンクをキューに追加
    for link in links:
        if link not in visited_urls and link not in url_queue:
             url_queue.append(link)

    # 進行状況の表示 (50ページごと)
    if len(site_map) % 50 == 0 and len(site_map) > 0:
        print(f"\n=========================================================")
        print(f"📈 進行状況: 現在までに {len(site_map)} ページを抽出しました。")
        print(f"📦 未探索URLキューのサイズ: {len(url_queue)}")
        print(f"=========================================================")


print("\n=========================================================")
print(f"🎉 クローリング終了。合計 {len(site_map)} ページを抽出しました。")
print(f"💡 終了理由: 未探索の同一ドメインURLがなくなりました。")
print("=========================================================")

# 最終結果の表示
print("\n=== 📚 抽出された武蔵野大学サイトマップ (URLとタイトル) ===")
# 辞書型変数を print() で表示
print(site_map)

🚀 クローリング開始: ベースURL https://www.musashino-u.ac.jp/
⏰ ディレイ時間: 1.0秒
🔍 ページ制限: 制限なし (理論上 1000000 ページまで)

-> アクセス中 (1ページ目): https://www.musashino-u.ac.jp/
--- ✅ 抽出完了: タイトル='武蔵野大学'

-> アクセス中 (2ページ目): https://www.musashino-u.ac.jp/access.html
--- ✅ 抽出完了: タイトル='交通アクセス | 武蔵野大学'

-> アクセス中 (3ページ目): https://www.musashino-u.ac.jp/admission/request.html
--- ✅ 抽出完了: タイトル='資料請求 | 入試情報 | 武蔵野大学'

-> アクセス中 (4ページ目): https://www.musashino-u.ac.jp/contact.html
--- ✅ 抽出完了: タイトル='お問い合わせ | 武蔵野大学'

-> アクセス中 (5ページ目): https://www.musashino-u.ac.jp/prospective-students.html
--- ✅ 抽出完了: タイトル='武蔵野大学で学びたい方 | 武蔵野大学'

-> アクセス中 (6ページ目): https://www.musashino-u.ac.jp/students.html
--- ✅ 抽出完了: タイトル='在学生の方 | 武蔵野大学'

-> アクセス中 (7ページ目): https://www.musashino-u.ac.jp/alumni.html
--- ✅ 抽出完了: タイトル='卒業生の方 | 武蔵野大学'

-> アクセス中 (8ページ目): https://www.musashino-u.ac.jp/parents.html
--- ✅ 抽出完了: タイトル='保護者の方 | 武蔵野大学'

-> アクセス中 (9ページ目): https://www.musashino-u.ac.jp/business.html
--- ✅ 抽出完了: タイトル='企業・研究者の方 | 武蔵野大学'

-> アクセス中 (10ページ目): https:

In [ ]:
j